In [5]:
import pandas as pd
import tkinter as tk
from tkinter import ttk
import requests
from io import StringIO
from unidecode import unidecode

df = None
tree = None
colunas_selecionadas = []
filter_comboboxes = {}
colunas = []

def determinar_delimitador(content):
    delimiters = [',', ';', '\t']
    for delimiter in delimiters:
        if delimiter in content:
            return delimiter
    return ','

def remover_acentos(text):
    return unidecode(text)

def carregar_arquivo():
    global df, colunas
    url = entrada_url.get()

    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Erro ao carregar o arquivo da web. Código de status: {response.status_code}")
        return

    content = response.text

    delimitador = determinar_delimitador(content)

    try:
        df = pd.read_csv(StringIO(content), sep=delimitador, on_bad_lines='skip', dtype=str)
    except Exception as e:
        print(f"Erro ao carregar o arquivo: {str(e)}")
        return

    df.columns = df.columns.map(remover_acentos)
    
    for col in df.columns:
        df[col] = df[col].apply(remover_acentos)
        df[col] = df[col].str.upper()
    
    colunas = df.columns.tolist()
    
    mensagem_status.config(text="Pronto para Carregar Colunas")
    botao_carregar_colunas.config(state=tk.NORMAL)
    entrada_url.config(state=tk.DISABLED)

def mostrar_tabela():
    if df is not None:
        tree.delete(*tree.get_children())  # Limpa a árvore

        if colunas_selecionadas:
            for index, row in df.iterrows():
                values = [row[col] for col in colunas_selecionadas if col in df.columns]
                tree.insert("", "end", values=values)

def carregar_colunas():
    global colunas_selecionadas
    colunas_selecionadas = entrada_colunas.get().split(',')

    if not colunas_selecionadas or colunas_selecionadas == ['']:
        print("Colunas não especificadas. Mostrando as quatro primeiras colunas.")
        colunas_selecionadas = [0, 1, 2, 3]  # Índices das quatro primeiras colunas
    else:
        # Converte os índices especificados pelo usuário em inteiros
        colunas_selecionadas = [int(idx) for idx in colunas_selecionadas]

    mensagem_status.config(text="Pronto para Mostrar Tabela")
    botao_mostrar_tabela.config(state=tk.NORMAL)
    entrada_colunas.config(state=tk.DISABLED)

def criar_filtros():
    if df is not None:
        global filter_comboboxes
        filter_comboboxes = {}
        frame_filtros.destroy()

        frame_filtros = tk.Frame(root)
        frame_filtros.grid(row=2, column=0, padx=10, pady=5, sticky="w")

        for col in colunas:
            valores_unicos = df[col].unique()
            valores_unicos = [str(val) for val in valores_unicos]
            valores_unicos.insert(0, "Todas")

            filtro = ttk.Combobox(frame_filtros, values=valores_unicos)
            filtro.set("Todas")
            filtro.grid(row=0, column=len(filter_comboboxes), padx=5)
            filter_comboboxes[col] = filtro

        botao_filtrar = tk.Button(frame_filtros, text="Filtrar Tabela", command=filtrar_tabela)
        botao_filtrar.grid(row=0, column=len(filter_comboboxes) + 1, padx=5)

        botao_reset = tk.Button(frame_filtros, text="Limpar Filtros", command=mostrar_tabela)
        botao_reset.grid(row=0, column=len(filter_comboboxes) + 2, padx=5)

def filtrar_tabela():
    global df
    filters = {}
    for col, combo in filter_comboboxes.items():
        selected_value = combo.get()
        if selected_value != "Todas":
            filters[col] = selected_value
    if filters:
        df = df[
            df.apply(
                lambda row: all(row[col] == filters[col] for col in filters),
                axis=1,
            )
        ]
    mostrar_tabela()

def sort_column(tree, col, reverse):
    l = [(tree.set(k, col), k) for k in tree.get_children('')]
    l.sort(reverse=reverse)
    for index, (val, k) in enumerate(l):
        tree.move(k, '', index)
    tree.heading(col, command=lambda: sort_column(tree, col, not reverse))

root = tk.Tk()
root.title("Leitor de CSV Online")

frame_url = tk.Frame(root)
frame_url.grid(row=0, column=0, padx=10, pady=5, sticky="w")
label_url = tk.Label(frame_url, text="URL do CSV:")
label_url.grid(row=0, column=0)
entrada_url = tk.Entry(frame_url)
entrada_url.grid(row=0, column=1)

botao_carregar = tk.Button(frame_url, text="Carregar Arquivo", command=carregar_arquivo)
botao_carregar.grid(row=0, column=2, padx=5)

mensagem_status = tk.Label(root, text="", padx=10, pady=5)
mensagem_status.grid(row=1, column=0)

frame_colunas = tk.Frame(root)
frame_colunas.grid(row=2, column=0, padx=10, pady=5, sticky="w")
label_colunas = tk.Label(frame_colunas, text="Colunas (separadas por vírgula):")
label_colunas.grid(row=0, column=0)
entrada_colunas = tk.Entry(frame_colunas)
entrada_colunas.grid(row=0, column=1)

botao_carregar_colunas = tk.Button(frame_colunas, text="Carregar Colunas", command=carregar_colunas, state=tk.DISABLED)
botao_carregar_colunas.grid(row=0, column=2, padx=5)

botao_mostrar_tabela = tk.Button(frame_colunas, text="Mostrar Tabela", command=mostrar_tabela, state=tk.DISABLED)
botao_mostrar_tabela.grid(row=1, column=1, padx=5)

frame_filtros = tk.Frame(root)
frame_filtros.grid(row=3, column=0, padx=10, pady=5, sticky="w")

# botao_criar_filtros = tk.Button(frame_filtros, text="Criar Filtros", command=criar_filtros, state=tk.DISABLED)
# botao_criar_filtros.grid(row=0, column=0, padx=5)

tree_frame = ttk.Frame(root)
tree_frame.grid(row=4, column=0, padx=10, pady=10, sticky="nsew")
tree_frame.grid_rowconfigure(0, weight=1)
tree_frame.grid_columnconfigure(0, weight=1)

canvas = tk.Canvas(tree_frame)
canvas.grid(row=0, column=0, sticky="nsew")

vsb = ttk.Scrollbar(tree_frame, orient="vertical", command=canvas.yview)
vsb.grid(row=0, column=1, sticky="ns")
canvas.configure(yscrollcommand=vsb.set)

hsb = ttk.Scrollbar(tree_frame, orient="horizontal", command=canvas.xview)
hsb.grid(row=1, column=0, sticky="ew")
canvas.configure(xscrollcommand=hsb.set)

root.columnconfigure(0, weight=1)
root.rowconfigure(4, weight=1)

tree = ttk.Treeview(canvas, columns=colunas, show="headings")

for col in colunas:
    tree.heading(col, text=col, command=lambda c=col: sort_column(tree, c, False))

tree.grid(row=0, column=0, sticky="nsew")

canvas.create_window((0, 0), window=tree, anchor="nw")
canvas.update_idletasks()
canvas.configure(scrollregion=canvas.bbox("all"))

root.mainloop()

Colunas não especificadas. Mostrando as quatro primeiras colunas.
